## Download code and data

In [ ]:
!git clone  https://github.com/dtungpka/AligningVTVSignlanguage.git .

If on colab, run the following cell to download the code and data. If running locally, you can download the data from [skeleton.zip](https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/EeqGCq4MsZtMm54TiKhx980BAPQSrwMjPX_hpMRaSMD-Uw?download=1) and [evaluation_pack.pkl](https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/ESIr97SZa6FNtG3P76ddu9EBk_eKCnVG-HHM71rjNVjKpQ?download=1)

In [ ]:
!wget -O skeletons.zip "https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/EeqGCq4MsZtMm54TiKhx980BAPQSrwMjPX_hpMRaSMD-Uw?download=1"
!unzip skeletons.zip
!wget -O evaluation_pack.pkl "https://daihocphenikaa-my.sharepoint.com/:u:/g/personal/21010294_st_phenikaa-uni_edu_vn/ESIr97SZa6FNtG3P76ddu9EBk_eKCnVG-HHM71rjNVjKpQ?download=1"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   199  100   199    0     0    361      0 --:--:-- --:--:-- --:--:--   363

100    13  100    13    0     0     16      0 --:--:-- --:--:-- --:--:--    16
tar: Error opening archive: Unrecognized archive format
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import os
import train
import pandas as pd
import os
import glob
import pickle

## 1. Select signs

In [ ]:
from select_signs import main as select_signs_main
if not os.path.exists('pending_training.txt'):
    select_signs_main()
else:
    print('pending_training.txt exists. Skip select_signs')

You should download the `pending_training.txt` file to backup it

## 2. Training

In [5]:
EPOCHS = 10
N_FRAMES = 5


signs_file = 'pending_training.txt'
if not os.path.exists(signs_file):
    print(f"File '{signs_file}' does not exist.")
    exit(1)
with open(signs_file, 'r') as f:
    signs_list = [line.strip() for line in f.readlines()]
train.process_signs(data_folder="skeletons", save_path="models", signs_list=signs_list, epochs=EPOCHS, n_frames=N_FRAMES)

Training signs from the provided list.


Training signs:   0%|          | 0/1 [00:00<?, ?it/s]

Training a-8851:   0%|          | 0/10 [00:00<?, ?it/s]

c:\Users\duong\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\duong\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\duong\AppData\Local\Programs\Python\Python39\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\duong\AppData\Local\Programs\Python\Python39\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\duong\AppData\Local\Programs\Python\Python39\lib\subproces

Accuracy: 0.8671875, Precision: 0.92, Recall: 0.6052631578947368, F1 Score: 0.7301587301587301
Performance report saved to results\training_results_49_12_14_5_11_2024.csv


In [6]:
# Get list of all CSV files in the results folder
csv_files = glob.glob(os.path.join('results', 'training_results_*.csv'))

print(csv_files)
# Get the newest CSV file
newest_csv = max(csv_files, key=os.path.getctime)

# Read the newest CSV file into a pandas DataFrame
df_newest = pd.read_csv(newest_csv)

# Display the DataFrame interactively
df_newest

['results\\training_results_14_8_14_5_11_2024.csv', 'results\\training_results_49_12_14_5_11_2024.csv']


,sign,accuracy,precision,recall,f1
0,a-8851,0.867188,0.92,0.605263,0.730159


## 3. Evaluate on VTV

In [ ]:
if os.path.exists(os.path.join('results', 'results.csv')):
    os.remove(os.path.join('results', 'results.csv'))
from evaluate import process_evaluation_pack

evaluation_pack_path = 'evaluation_pack.pkl'
checkpoint_path = 'models'
save_path = 'results'
frame_count = 5 #equal to N_FRAMES
delta = 120 #window size to find the sign

# Load evaluation pack and process
with open(evaluation_pack_path, 'rb') as f:
    evaluation_pack = pickle.load(f)

process_evaluation_pack(evaluation_pack, checkpoint_path, save_path, frame_count, delta)

In [7]:
# Get list of all CSV files in the results folder
csv_files = glob.glob(os.path.join('results', 'results.csv'))

print(csv_files)
# Get the newest CSV file
newest_csv = max(csv_files, key=os.path.getctime)

# Read the newest CSV file into a pandas DataFrame
df_newest = pd.read_csv(newest_csv)

# Display the DataFrame interactively
df_newest

['results\\results.csv']


,word,video,model,iou,mAP,f1,threshold
0,8,VNHN_16_06_24_cropped_part2,8-9627_84_70.pth,0.601329,0.755292,0.760622,0.0
1,12,VNHN_05_03_24_cropped_part3,12-9633_87_73.pth,0.586207,0.656219,0.749186,0.0
2,12,VNHN_05_03_24_cropped_part3,12-9634_86_74.pth,0.586207,0.720249,0.749186,0.0
3,12,VNHN_08_04_24_cropped_part3,12-9633_87_73.pth,0.581152,0.653937,0.745314,0.0
4,12,VNHN_08_04_24_cropped_part3,12-9634_86_74.pth,0.581152,0.674817,0.745314,0.0
5,12,VNHN_08_06_24_cropped_part2,12-9633_87_73.pth,0.605263,0.659109,0.763562,0.0
6,12,VNHN_08_06_24_cropped_part2,12-9634_86_74.pth,0.605263,0.694548,0.763562,0.0
7,12,VNHN_08_06_24_cropped_part4,12-9633_87_73.pth,0.617834,0.688295,0.772861,0.0
8,12,VNHN_08_06_24_cropped_part4,12-9634_86_74.pth,0.617834,0.641132,0.772861,0.0
9,12,VNHN_16_06_24_cropped_part2,12-9633_87_73.pth,0.575221,0.720797,0.740741,0.0
